# Таксономия алгоритмов обучения с подкрепением

![taxonomy](images/taxonomy.png "Taxonomy")

# Imitation learning

 Основной проблемой данного подхода является то, что даже при небольших отклонениях от оригинальной траектории агент попадает в ситуацию, с которой никогда до этого не сталкивался. Эта проблема имеет название Data Mismatch problem

![dagger](images/dagger.png "DAgger")

# DAgger

DAgger ( Dataset Aggregation ) - это алогритм, который позволяет обучать политику с помощью алгоритмов обучения с учителем.

![dagger-algo](images/dagger-algo.png "DAgger algorithm")

#  DAgger одновременный сбор данных

Другой идеей может быть одновременное получение выборки с отклонениями от основной траектории. В качестве примерра можно привести управление автомобилем или дроном, когда сбор данных производится одновременно с нескольких камер, при том, что одна из которых смотрит прямо, а две установленны под некоторым углом, чтобы симулировать ситуацию, когда агент отклонился от основной траектории

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/voVxIGyeqgo?rel=0" frameborder="0" gesture="media" allow="encrypted-media" allowfullscreen></iframe>

![nvidia-rig](images/forest-rig.png "Nvidia Rig")

Машинное обучение с применением лома

# Policy Gradient

Одним из наиболее просты подходов к решению задач обучения с подкреплением является прямая оптимизация политики методом градиентного спуска. Мы параметризуем политику π набором параметров $\theta$ и пытаемся найти значение $\theta^*$:

$\theta^* = \text{arg}\max\limits_{\theta}\, E_{r \sim p_θ(τ)}  [ \sum\limits_{t} r(s_t, a_t) ]$


# REINFORCE

Одним из старейших алгоритмов данного семейства является REINFORCE. Он выводится напрямую:

1) В качестве функции потерь берётся:

$$J(\theta) = \int\limits_{\mathbb T} p_{\theta}(\tau|\pi)R(\tau)d{\tau}$$

2) Её градиент:

$$ \nabla_{\theta} J(\theta) = \nabla_\theta \int\limits_{\mathbb T} p_\theta(\tau | \pi) R(\tau)d\tau $$

3) Подставляя:

$$ \nabla_\theta p_\theta(\tau|\pi) = p_\theta(\tau|\pi) \nabla_\theta \log p_\theta(\tau|\pi)$$

Получаем:

$$J(\theta) = \int\limits_{\mathbb T} p_\theta(\tau|\pi) \nabla \log p_\theta(\tau|\pi) R(\tau) d\tau$$
$$= E(\nabla_\theta \log p_\theta(\tau|\pi) R(\tau))$$
$$\approx \dfrac{1}{K} \sum\limits_{k=1}^K \nabla_\theta \log p_\theta(\tau_k|\pi)R(\tau_k)$$

Используя последнее выражение мы можем оптимизировать политику итеративно по сэмплам полученным в результате симуляции.

REINFORCE - это акроним REward Increment = Nonnegative Factor \* Offset Reinforcement \* Characteristic Eligibility, что описывает шаг алгоритма. На каждом шаге мы обновляем веса политики


# Policy Gradient vs Maximum Likelihood

![polgrad_maxlike](images/polgrad_maxlikelihood.png "Polgrad vs Maxlike")

# Policy Gradient

Преимущества:

* Хорошо сходится
* Может эффективно работать в пространствах действий высокой размерности или с непрерывными действиями 
* Можно задействовать экспертную демонстрацию
* Мы имеем прямой контроль над исследованием среды
* Позволяет обучать стохаистические политики

Недостатки:

* Обычно сходится к локальному, а не глобальному минимуму
* Высокая дисперсия результатов выполнения политики

# Policy Gradient. Пример

Atari Pong

* 8000 эпизодов по ~30 игр
* Policy network - двуслойная полносвязная архитектура (пиксели)->( 200 скрытых параметров ) -> (действие) 
* RMSProp lr=1e-4

In [4]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/YOW8m2YGtRg?rel=0" frameborder="0" gesture="media" allow="encrypted-media" allowfullscreen></iframe>

# Value based

Мы можем подойти к задаче обучения с подкреплением с другой стороны. Если мы можем предсказать ценность ( сумму наград до конца игры ) каждого состояния или пары состояние-действие, то мы тривиально можем построить оптимальную политику каждый раз выбирая действие, которое либо имеет наибольшую ценность, либо приведёт нас в состояние с наибольшей ценностью.  

# SARSA

Одним из наиболее простых Value-based методом является SARSA. В данном методе мы пытаемся выучить ценность действия в заданном состоянии ( action value ). Обучение происходит в режиме on-policy, то есть мы используем одну и ту же политику для исследования и управления. На каждом шаге мы обновляем значения ценности:

$$ Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha [ r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)], где $$

$\alpha$ - learning rate

$\gamma$ - дисконтирующий фактор 

Для обучения мы используем кортеж $ ( s_t, a_t, r_t, s_{t+1}, a_{t+1} ) $ - отсюда название алгоритма.

# Q-learning ( table based )

Если мы решим использовать отдельную политику для исследования среды ( учиться в режиме off-policy ) то мы столкнёмся с проблемой - ценность следующего действия теперь зависит от неоптимальной политики, и вместо $Q_{\pi^*}$ ( ценностью действия при оптимальной политике ) мы выучиваем $Q_{\pi_{exploratory}}$. Эта проблема может быть легко решена, если для обновления значения Q мы вместо того действия, которое политика избрала в реальности возьмём действие, которое является оптимальным с точки зрения политики. 

Полученный алгоритм будет называться Q-learning и может эффективно применяется для задач с небольшим дискретным пространством действий.

# DQN

В 2013 году небольшой и малоизвестной компанией DeepMind была опубликованна статья описывающая применение нейронной сети для аппроксимации значения Q и обучения агента играм Atari. Авторам статьи удалось добиться результатов превосходящих человеческие на существенной доле игр используя только сырые значения пикселей и очки из игр.

До того момента value-based методы с применением нейронных сетей считались слишком нестабильными для практического применения. Авторы статьи применили несколько методов для того, чтобы улучшить сходимость:

1. При последовательном обучении модели на результатах симуляции данные в батчах обучающей выборки оказываются сильно скоррелированы. Решением этой проблемы является введение специального кольцевого буфера, в котором хранятся тренировочные данные, и из которого случайным образом сэмплятся батчи для тренировки. Данная техника называется experience replay

2. Из-за особенностей алгоритма, при использовании одной и той же сети для пердсказания и управления происходит амплификация шума. Для решения этой проблемы сеть периодически клонируется в так называемую target network, которая и используется для управления агентом. Это позволяет декоррелировать шум и улучшает сходимость.

3. Для того, чтобы упростить обучение в качестве тензора состояния использовались несколько последовательных кадров с последующей 3D свёрткой по ним. Это позволяет сети выделять динамические признаки и сильно ускоряет обучение. Вместо этого подхода часто используют RNN

4. Стандартной политикой во время тренировки является Annealed ε-greedy policy. Это означает, что агент с вероятностью 1-ε выбирает действие с максимальным предсказанным Q, а с вероятностью ε - случайное действие. При этом сам параметр ε убывает с течением времени по заранее заданному расписанию  

# Расширения DQN

После публикации оригинальной статьи было предложено несколько расширений оригинального алгоритма:

### Double DQN

Существенной пробемой DQN является то, что Qnetwork имеет тенденцию переоценивать значения Q. Это происходит из-за того, что для оценки Q использется max в:

$$ Q(s, a) \xrightarrow{} r + \gamma max_a Q(s', a) $$

Для решения этой проблемы можно тренировать две сети, и использовать одну из них для выбора действия, а другую для оценки его значения. При этом, так как мы уже тренируем две сети ( основную и target ) часто для реализации DDQN достаточно просто использовать target network для выбора действия

### Deep Attention Q-learning network

Во многих задачах добавление механизма attention позволяет улучшить результаты. Для этого мы тренируем отдельную небольшую модель, которая определяет, какие области входных данных являются релевантными - после чего мы используем полченную маску для того, чтобы основная сеть концентрировалась на релевантных для задачи участках. В качестве бонуса мы получаем лёгкий способ визуализации работы сети, который позволяет нам легко понять, какие признаки сеть использует для предсказания

### Prioritized Samling

Одним из способов улучшить скорость обучения алгоритма - это приотетичированный сэмплинг из replay buffer. Мы увеличиваем верояность сэмплинга тех состояний, на которых ошибка максимальна.

### Dueling DQN

Идея Dueling DQN заключается в том, что мы декомпозируем action-value Q(s, a) на value V(s) и advantage A(s, a). Это осуществляется архетиктурой, в которой Qnetwork имеет две головы - одна из них считает V(s) пользуясь тем, что эта часть не зависит от действия, а вторая голова считает A(s, a), пользуясь тем, что $\sum\limits_a A(s, a) = 0$


# Actor Critic

Большое распространение в RL получила гибридная архитектура Actor-Critic. Методы данного класса совмещают в себе Policy Gradient и предсказание ценности действий, при этом полученные политики имеют существенно более низкую дисперсию, потому, что для предсказания ценности действия используется более одного примера

![actorcritic](images/actor-critic.png "Actor Critic")

# DDPG

После успеха DQN была предложена архитектура DDPG, которая использует основные идеи из DQN в Actor-Critic архитектуре, и позволяет использовать Deep Q-network для управления агентом в задачах с непрерывным пространством действий. 

# Теорема о детерменистическом градиенте политики 

Авторы статьи Determenistic Policy Gradient Algorithms показали, что градиент детерминистической политики может быть рассчитан, и что он равен матожиданию градиента функции ценности действия ( Q, action-value function)

$$ \nabla_\theta(\mu_\theta) = \int\limits_S \rho^\mu(s) \nabla_\theta \mu_\theta(s) \nabla_a Q^\mu(s, a)\space\rvert_{a=\mu_\theta(s)} ds  $$

$$ = \mathbb E_{s\sim\rho^\mu} [ \nabla_\theta\mu_\theta(s) \nabla_a Q^\mu(s, a)\rvert_{a=\mu_\theta(s)} ] $$

Где $\mu_\theta$ - детерминистическая политика, $\theta$ - параметры политики


# DDPG

Пользуясь этим результатом авторы статьи про DDPG реализовали Q-learning network в полном соответствии с DQN, и интегрировали её в классический DQN алгоритм.

# a3c

Другим, в крайней степени популярным AC методом является a3c. Название метода расшифровывается как Asynchronous Advantage Actor-Critic. В данном методе:

1. Несколько независимых агентов *асинхронно* взаимодействуют с окружением ( каждый со своей копией ) и обновляют параметры собственной копии сети. Это позволяет более оптимально собирать данные, а также автоматически декоррелирует тренировочную выборку

2. Мы обучаем сеть предсказывать значение фунцкии A, по аналогии с Dueling DQN

3. Мы обновляем веса основной сети двигая из в сторону весов обученных сетей на стороне воркеров по распсанию

4. Периодически мы подставляем веса основной сети в воркеров

# Model based methods

* Sample efficient
* Transferable
* Generally have worse performance than model-free methods


# Оптимальный контроль и планирование 

Во многих случаях у нас есть модель в которой действует агент. К примеру:

* Игры ( шахматы, Го )
* Физические эмуляторы
* Предсказуемые системы ( к примеру, автомобиль )

В таком случае мы можем воспользоваться алгоритмами из теории оптимального управления

# Optimal Control Methods

* iRLC
* Monte-Carlo Tree Search

# Выучивание модели

* Параметризированная физика
* Обучение напрямую из картинок

# Curiosity Based Exploration